In [ ]:
!nvidia-smi ## GPU 정보보기 ( 실시간 )
## Tesla K80 : GPU 이름
## watch nvidia-smi

In [ ]:
!export CUDA_VISIBLE_DEVICES=2 ## 5번 GPU 사용 (모델이 깊어지면 지정하여 전체 사용GPU 사용분산)

In [ ]:
## CNN 모델사용 

In [10]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt


tf.reset_default_graph() : 주피터 노트북과 같은 환경에서 실행을 하게 되면, 주피터 커널을 리스타트하지 않는 이상 변수들의 컨택스트가 그대로 유지 되기 때문에, 위의 코드를 같은 커널에서 tf.reset_default_graph() 없이, 두 번 이상 실행하게 되면 에러가 난다. 그 이유는 텐서플로우 그래프를 만들어놓고, 그 그래프가 지워지지 않은 상태에서 다시 같은 그래프를 생성하면서 나오는 에러인데, tf.reset_default_graph() 메서드는 기존에 생성된 디폴트 그래프를 모두 삭제해서 그래프가 중복되는 것을 막아준다. 

In [11]:
tf.reset_default_graph()

def parser(serialized_example):
    feature = {
        'age': tf.FixedLenFeature([1], tf.int64),#Configuration for parsing a fixed-length input feature.
        'img': tf.FixedLenFeature([61*49], tf.int64) ## 61*49 픽셀
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, feature)
    age = tf.cast(parsed_feature['age'], tf.int32)## one-hot 으로 바꿔야 하므로 intiger
    img = tf.cast(parsed_feature['img'], tf.float32)## imgage는 실수형
    
    return age, img

In [12]:
face_train = './cnn_dataset/face_train.tfrecord'
face_test = './cnn_dataset/face_test.tfrecord'
train_dataset = tf.contrib.data.TFRecordDataset(face_train).map(parser)## parcer fuction mapping 
train_dataset = train_dataset.batch(32).shuffle(7777) 

In [13]:
test_dataset = tf.contrib.data.TFRecordDataset(face_test).map(parser)
test_dataset = test_dataset.batch(32).shuffle(7777)


In [ ]:
itr = train_dataset.make_one_shot_iterator() ## iterator 생성

age, img = itr.get_next()## 하나끝나면 그다음꺼 하나씩 빼옴 

In [ ]:
## one_shot_iterator 문제점: trainset, testset의 크기가 다른경우 
## out of range 에러가 생기는 경우 
img = tf.reshape(img,[-1,61,49,1]) ## 자동셋팅, 행61 * 열49 pixel, 1color(흑백) 
img = tf.cast(img, tf.float32) ## 실수형 32 변경 

In [ ]:
age = tf.reshape(age,[-1]) ## 자동셋팅, 일자로 쫙펴줌 ## reduce_mean 써도됨
age = tf.one_hot(age, depth = 3, axis = -1, dtype = tf.float32)


In [ ]:
with tf.Session() as sess:
    _age,_img = sess.run([age, img])
    print(_age)
    print(_img)
    # plt.imshow(_img)

In [ ]:
## convolution 시작 

In [ ]:
activation = tf.nn.relu ## convolution : relu function 
reuse = False

In [ ]:
conv1 = tf.layers.conv2d(img,filters=16,
                         kernel_size = 4, padding='same', 
                         activation = activation, reuse =reuse, 
                         name = 'conv1')

print(conv1.shape)
pool1 = tf.layers.max_pooling2d(conv1, pool_size = 2, strides =2) ## 사이즈2, 보폭2 인 polling
print(pool1.shape)

In [ ]:
conv2= tf.layers.conv2d(conv1, filters=32,
                       kernel_size = 3, padding='same', 
                        activation = activation, reuse =reuse, 
                         name = 'conv2')
print(conv2.shape)
pool2 = tf.layers.max_pooling2d(conv2,pool_size = 2, strides =2)
print(pool2.shape) ## shape 가 홀수인경우 pooling 하면 나머지줄은 버림 

In [ ]:
conv3= tf.layers.conv2d(conv2, filters=32,
                       kernel_size = 4, padding='same', 
                        activation = activation, reuse =reuse, 
                         name = 'conv3')
print(conv3.shape)
pool3 = tf.layers.max_pooling2d(conv3,pool_size = 2, strides =2)

print(pool3.shape)

In [ ]:
## layer 시작 세팅 
flat = tf.layers.flatten(pool3)
## flat_size = int(pool3.shape[1]) * int(pool3.shape[2]) * int(pool3.shape[3])
## flat = tf.reshape(pool3, [-1, flat_size]) 으로 대체가능 
print(flat.shape)

In [ ]:
## dropout 설정

In [ ]:
fc1 = tf.layers.dense(flat, units = 5000, 
                      activation = tf.nn.relu, 
                      reuse =reuse, name = 'fc1')

print(fc1.shape)

In [ ]:
fc2 = tf.layers.dense(fc1, units = 1000, 
                      activation = tf.nn.relu, 
                      reuse =reuse, name = 'fc2')
print(fc2.shape)

In [ ]:
out = tf.layers.dense(fc2, units = 3, 
                      activation = tf.nn.relu, 
                      reuse =reuse, name = 'out')
print(out.shape)

## 강사님 코드 

import tensorflow as tf

tf.reset_default_graph()

def parser(serialized_example):
    features = {
        'age': tf.FixedLenFeature([1], tf.int64),
        'img': tf.FixedLenFeature([61*49], tf.int64)
    }
    
    parsed_feature = tf.parse_single_example(serialized_example, features)
    age = parsed_feature['age']
    img = parsed_feature['img']
    return age, img


train_data_dir = '../cnn_dataset/face_train.tfrecord'
test_data_dir = '../cnn_dataset/face_test.tfrecord'

train_dataset = tf.contrib.data.TFRecordDataset(train_data_dir).map(parser)
train_dataset = train_dataset.shuffle(7777)
train_dataset = train_dataset.batch(32)

itr = train_dataset.make_one_shot_iterator()

age, img = itr.get_next()

img = tf.reshape(img, [-1, 61, 49, 1])
img = tf.cast(img, tf.float32)

age = tf.reshape(age, [-1])
age = tf.one_hot(age, depth=3, axis=-1, dtype=tf.float32)

activation = tf.nn.relu
reuse = False
conv1 = tf.layers.conv2d(img, 
                         filters=16, 
                         kernel_size=4, 
                         padding='same', 
                         activation=activation, 
                         reuse=reuse, 
                         name='conv1')
print(conv1.shape)
conv2 = tf.layers.conv2d(conv1,
                        filters=32,
                        kernel_size=3,
                        padding='same',
                        activation=activation,
                        reuse=reuse,
                        name='conv2')
print(conv2.shape)
pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)
print(pool2.shape)

conv3 = tf.layers.conv2d(pool2,
                        filters=64,
                        kernel_size=3,
                        padding='same',
                        activation=activation,
                        reuse=reuse,
                        name='conv3')
print(conv3.shape)
pool3 = tf.layers.max_pooling2d(conv3, pool_size=2, strides=2)
print(pool3.shape)

flat = tf.layers.flatten(pool3)
print(flat.shape)

fc1 = tf.layers.dense(flat, 
                      units=5000, 
                      activation=tf.nn.relu, 
                      reuse=reuse, 
                      name='fc1')

print(fc1.shape)
fc2 = tf.layers.dense(fc1,
                     units=1000,
                     activation=tf.nn.relu,
                     reuse=reuse,
                     name='fc2')
print(fc2.shape)

out = tf.layers.dense(fc2,
                     units=3,
                     reuse=reuse,
                     name='out')
print(out.shape)

# with tf.Session() as sess:
#     _age, _img = sess.run([age, img])